# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here

import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
cur = conn.cursor()
# Execute the query
# (This is a special query for finding the table names. You don't need to memorize it.)
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
# Fetch the result and store it in table_names
table_names = cur.fetchall()
table_names

[('productlines',),
 ('offices',),
 ('customers',),
 ('employees',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('products',)]

In [3]:
cur.execute("""SELECT * FROM offices;""")
cur.fetchall()


[(1,
  'San Francisco',
  '+1 650 219 4782',
  '100 Market Street',
  'Suite 300',
  'CA',
  'USA',
  '94080',
  'NA'),
 (2,
  'Boston',
  '+1 215 837 0825',
  '1550 Court Place',
  'Suite 102',
  'MA',
  'USA',
  '02107',
  'NA'),
 (3,
  'NYC',
  '+1 212 555 3000',
  '523 East 53rd Street',
  'apt. 5A',
  'NY',
  'USA',
  '10022',
  'NA'),
 (4,
  'Paris',
  '+33 14 723 4404',
  "43 Rue Jouffroy D'abbans",
  '',
  '',
  'France',
  '75017',
  'EMEA'),
 (5,
  'Tokyo',
  '+81 33 224 5000',
  '4-1 Kioicho',
  '',
  'Chiyoda-Ku',
  'Japan',
  '102-8578',
  'Japan'),
 (6,
  'Sydney',
  '+61 2 9264 2451',
  '5-11 Wentworth Avenue',
  'Floor #2',
  '',
  'Australia',
  'NSW 2010',
  'APAC'),
 (7,
  'London',
  '+44 20 7877 2041',
  '25 Old Broad Street',
  'Level 7',
  '',
  'UK',
  'EC2N 1HN',
  'EMEA'),
 (27,
  'Boston',
  '+1 977 299 8345',
  '105 Cambridge Street',
  '',
  'MA',
  'USA',
  '02331',
  'NA')]

In [4]:
cur.description

(('officeCode', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('phone', None, None, None, None, None, None),
 ('addressLine1', None, None, None, None, None, None),
 ('addressLine2', None, None, None, None, None, None),
 ('state', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('postalCode', None, None, None, None, None, None),
 ('territory', None, None, None, None, None, None))

In [5]:
cur.execute("""SELECT * FROM employees;""")
cur.fetchall()


[(1002,
  'Murphy',
  'Diane',
  'x5800',
  'dmurphy@classicmodelcars.com',
  1,
  '',
  'President'),
 (1056,
  'Patterson',
  'Mary',
  'x4611',
  'mpatterso@classicmodelcars.com',
  1,
  1002,
  'VP Sales'),
 (1076,
  'Firrelli',
  'Jeff',
  'x9273',
  'jfirrelli@classicmodelcars.com',
  1,
  1002,
  'VP Marketing'),
 (1088,
  'Patterson',
  'William',
  'x4871',
  'wpatterson@classicmodelcars.com',
  6,
  1056,
  'Sales Manager (APAC)'),
 (1102,
  'Bondur',
  'Gerard',
  'x5408',
  'gbondur@classicmodelcars.com',
  4,
  1056,
  'Sale Manager (EMEA)'),
 (1143,
  'Bow',
  'Anthony',
  'x5428',
  'abow@classicmodelcars.com',
  1,
  1056,
  'Sales Manager (NA)'),
 (1165,
  'Jennings',
  'Leslie',
  'x3291',
  'ljennings@classicmodelcars.com',
  1,
  1143,
  'Sales Rep'),
 (1166,
  'Thompson',
  'Leslie',
  'x4065',
  'lthompson@classicmodelcars.com',
  1,
  1143,
  'Sales Rep'),
 (1188,
  'Firrelli',
  'Julie',
  'x2173',
  'jfirrelli@classicmodelcars.com',
  2,
  1143,
  'Sales Rep'),

In [6]:
cur.description

(('employeeNumber', None, None, None, None, None, None),
 ('lastName', None, None, None, None, None, None),
 ('firstName', None, None, None, None, None, None),
 ('extension', None, None, None, None, None, None),
 ('email', None, None, None, None, None, None),
 ('officeCode', None, None, None, None, None, None),
 ('reportsTo', None, None, None, None, None, None),
 ('jobTitle', None, None, None, None, None, None))

In [7]:
# Your code here, how do you know what to select if you can't see the column names? 
#Is there a way to fetch all of the tables and their column names?

q = """
SELECT firstName, lastName
FROM employees
JOIN offices
    USING(officeCode)
WHERE city = 'Boston';
"""
pd.read_sql(q,conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [8]:
# Your code here

# COUNT(*) doesn't work here because
# we are trying to count the _employees_ in each group.
# So instead we count by some attribute of an employee
# record. The primary key (employeeNumber) is a 
# conventional way to do this
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) AS n_employees
FROM offices AS o
LEFT JOIN employees AS e
    USING(officeCode)
GROUP BY officeCode
HAVING n_employees = 0
;
"""
pd.read_sql(q, conn)

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [9]:
# Answers will vary

# Example question: How many customers are there per office?

q= """
SELECT
o.officeCode,
o.city,
Count (c.customerNumber) AS n_customers

FROM offices AS o
JOIN employees AS e
USING(officeCode)
JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode;
"""

pd.read_sql(q, conn)


,officeCode,city,n_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [10]:
#What are the job titles of the employees at each office?

# Your code here
q = """
SELECT o.officeCode,
o.city,
GROUP_ConCAT (e.jobTitle, ', ') AS jobTitles


FROM offices AS o
JOIN employees AS e
ON o.officeCode = e.officeCode
GROUP BY o.officeCode, o.city;
"""
pd.read_sql(q,conn)


,officeCode,city,jobTitles
0,1,San Francisco,"President, Sales Manager (NA), Sales Rep, Sale..."
1,2,Boston,"Sales Rep, Sales Rep"
2,3,NYC,"Sales Rep, Sales Rep"
3,4,Paris,"Sale Manager (EMEA), Sales Rep, Sales Rep, Sal..."
4,5,Tokyo,"Sales Rep, Sales Rep"
5,6,Sydney,"Sales Manager (APAC), Sales Rep, Sales Rep, Sa..."
6,7,London,"Sales Rep, Sales Rep"


In [11]:
q="""
WITH RECURSIVE RecursiveJobTitles AS (
SELECT 1 AS row_num, o.officeCode AS officeCode, o.city, e.jobTitle
FROM offices AS o
JOIN employees AS e ON o.officeCode = e.officeCode
WHERE e.jobTitle IS NOT NULL
UNION ALL
SELECT row_num + 1, r.officeCode, r.city, NULL
FROM RecursiveJobTitles AS r
WHERE r.jobTitle IS NOT NULL
)
SELECT officeCode, city, jobTitle
FROM RecursiveJobTitles
WHERE jobTitle IS NOT NULL
ORDER BY row_num, officeCode;
"""
pd.read_sql(q,conn)

,officeCode,city,jobTitle
0,1,San Francisco,President
1,1,San Francisco,VP Sales
2,1,San Francisco,VP Marketing
3,1,San Francisco,Sales Manager (NA)
4,1,San Francisco,Sales Rep
5,1,San Francisco,Sales Rep
6,2,Boston,Sales Rep
7,2,Boston,Sales Rep
8,3,NYC,Sales Rep
9,3,NYC,Sales Rep


In [12]:
#Determine the number of sales that each sales representative 
#makes at each office

# Your code here

q="""
SELECT
    o.officeCode,
    o.city,
    e.employeeNumber AS salesRepEmployeeNumber,
    COUNT(o2.orderNumber) AS totalSales
FROM
    offices AS o
JOIN employees AS e ON o.officeCode = e.officeCode
LEFT JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
LEFT JOIN orders AS o2 ON c.customerNumber = o2.customerNumber
GROUP BY
    o.officeCode, o.city, e.employeeNumber;
"""
pd.read_sql(q,conn)

,officeCode,city,salesRepEmployeeNumber,totalSales
0,1,San Francisco,1002,0
1,1,San Francisco,1056,0
2,1,San Francisco,1076,0
3,1,San Francisco,1143,0
4,1,San Francisco,1165,34
5,1,San Francisco,1166,14
6,2,Boston,1188,14
7,2,Boston,1216,18
8,3,NYC,1286,17
9,3,NYC,1323,22


In [13]:
#how many sales are due to repeat customers?

# Your code here
q="""
SELECT
    o.officeCode,
    o.city,
    e.employeeNumber AS salesRepEmployeeNumber,
    COUNT(DISTINCT c.customerNumber) AS distinctCustomers,
    COUNT(o2.orderNumber) AS totalSales
FROM
    offices AS o
JOIN employees AS e ON o.officeCode = e.officeCode
LEFT JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
LEFT JOIN orders AS o2 ON c.customerNumber = o2.customerNumber
GROUP BY
    o.officeCode, o.city, e.employeeNumber;
"""

pd.read_sql(q,conn)

,officeCode,city,salesRepEmployeeNumber,distinctCustomers,totalSales
0,1,San Francisco,1002,0,0
1,1,San Francisco,1056,0,0
2,1,San Francisco,1076,0,0
3,1,San Francisco,1143,0,0
4,1,San Francisco,1165,6,34
5,1,San Francisco,1166,6,14
6,2,Boston,1188,6,14
7,2,Boston,1216,6,18
8,3,NYC,1286,7,17
9,3,NYC,1323,8,22


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [17]:
# Your code here


# We don't need to use aliases for the columns since they
# are conveniently already labeled as different kinds of
# names (firstName, lastName, productName)
q = """
SELECT firstName, lastName, productName
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products
    USING(productCode)
;
"""
df = pd.read_sql(q, conn)
df

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [18]:
# Your code here
q = """
SELECT firstName, lastName, SUM(quantityOrdered) as total_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,total_products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [19]:
# Your code here
# Recall that HAVING is used for filtering after an aggregation
q = """
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


In [20]:
conn.close()

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!